In [10]:
!pip install lxml polars

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/f6/69/0a8d491916221d158008146d38614e150ec94e61cd32747be14ef7ba4fa4/polars-0.19.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 10.3 MB/s eta 0:00:00m eta 0:00:010:00:01


In [23]:
from bs4 import BeautifulSoup
from os import getcwd
import polars as pl

current_folder=getcwd()
file_path=f"{current_folder}/airports.txt"

# Sample HTML content
with open(file_path) as file:
    html_content = file.read()

# Create a BeautifulSoup object to parse the HTML
soup = BeautifulSoup(html_content, 'lxml')

# Create a BeautifulSoup object to parse the HTML
soup = BeautifulSoup(html_content, 'lxml')

# Find all table rows (tr elements)
table_rows = soup.find_all('tr')

# Initialize empty lists for each column
cities = []
countries = []
symbols = []

# Loop through each table row
for row in table_rows:
    # Find all table cells (td elements) within the row
    cells = row.find_all('td')
    
    # Extract data from the cells
    if len(cells) == 3:  # Ensure there are exactly 3 cells
        city = cells[0].text.strip()
        country = cells[1].text.strip()
        symbol = cells[2].text.strip()
        
        # Append data to respective lists
        cities.append(city)
        countries.append(country)
        symbols.append(symbol)

# Create a Polars DataFrame
df = pl.DataFrame({
    "city": cities,
    "country": countries,
    "symbol": symbols
})

# Display the Polars DataFrame
print(df)

# Save the Polars DataFrame to a CSV file
df.write_csv("airports.csv")

shape: (1_579, 3)
┌────────────────┬─────────────┬────────┐
│ city           ┆ country     ┆ symbol │
│ ---            ┆ ---         ┆ ---    │
│ str            ┆ str         ┆ str    │
╞════════════════╪═════════════╪════════╡
│ Aalborg        ┆ Denmark     ┆ AAL    │
│ Aalesund       ┆ Norway      ┆ AES    │
│ Aarhus         ┆ Denmark     ┆ AAR    │
│ Abbotsford, BC ┆ Canada      ┆ YXX    │
│ …              ┆ …           ┆ …      │
│ Zaragoza       ┆ Spain       ┆ ZAZ    │
│ Zhengzhou      ┆ China       ┆ CGO    │
│ Zhoushan       ┆ China       ┆ HSN    │
│ Zurich         ┆ Switzerland ┆ ZRH    │
└────────────────┴─────────────┴────────┘


In [16]:
from numpy import unique

countries=unique(list(df['country']))

len(countries)

209

In [22]:
# Count occurrences by 'Country' column
counted_df = df.group_by('country').agg(pl.count('*'))

# Display the counted DataFrame
print(counted_df)

# Save the Polars DataFrame to a CSV file
counted_df.write_csv("airports_count.csv")

shape: (209, 3)
┌──────────────┬──────┬────────┐
│ country      ┆ city ┆ symbol │
│ ---          ┆ ---  ┆ ---    │
│ str          ┆ u32  ┆ u32    │
╞══════════════╪══════╪════════╡
│ Lebanon      ┆ 1    ┆ 1      │
│ Saudi Arabia ┆ 4    ┆ 4      │
│ Greece       ┆ 14   ┆ 14     │
│ England      ┆ 18   ┆ 18     │
│ …            ┆ …    ┆ …      │
│ USA          ┆ 446  ┆ 446    │
│ Reunion      ┆ 2    ┆ 2      │
│ Georgia      ┆ 1    ┆ 1      │
│ Croatia      ┆ 4    ┆ 4      │
└──────────────┴──────┴────────┘
